# Description du MDP et exemple d'utilisation 

In [1]:
using POMDPs
using QuickPOMDPs
using Distributions
using Random
using POMDPTools: Deterministic

const MAX_INVENTORY = 20
const MAX_STORE = 10
const ORDER_SIZE = 5

const holding_cost_store = 2
const holding_cost_parking = 4
const order_cost = 20
const stockout_penalty = 50

const demand_dist = DiscreteUniform(0, 10)

mdp = QuickMDP(
    
    states = 0:MAX_INVENTORY,
    actions = [0, 1],
    discount = 0.99,

    gen = function (s, a, rng)
        order_qty = a == 1 ? min(ORDER_SIZE, MAX_INVENTORY - s) : 0
        new_stock = s + order_qty

        d = rand(rng, demand_dist)
        sold = min(d, new_stock)
        sp = new_stock - sold

        lost_sales = max(d - new_stock, 0)

        in_store = min(sp, MAX_STORE)
        in_parking = max(sp - MAX_STORE, 0)

        cost = 0
        cost += a == 1 ? order_cost : 0
        cost += in_store * holding_cost_store
        cost += in_parking * holding_cost_parking
        cost += lost_sales * stockout_penalty

        r = -cost

        return (sp, r, d)
    end,
    initialstate = Deterministic(10)
)


QuickMDP{Base.UUID("16aa6965-baab-40b9-927d-f210292ddded"), Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, actionindex::Dict{Int64, Int64}, initialstate::Deterministic{Int64}, states::UnitRange{Int64}, actions::Vector{Int64}, discount::Float64, gen::var"#11#12"}}((stateindex = Dict(5 => 6, 16 => 17, 7 => 8, 20 => 21, 12 => 13, 8 => 9, 17 => 18, 1 => 2, 19 => 20, 0 => 1…), isterminal = false, actionindex = Dict(0 => 1, 1 => 2), initialstate = Deterministic{Int64}(10), states = 0:20, actions = [0, 1], discount = 0.99, gen = var"#11#12"()))

In [2]:
using Random
rng = MersenneTwister(42)
s = 7
a = 0
for i in 1:10
    sp, r, d = POMDPs.gen(mdp, s, a, rng)
    println("From $s --(a=$a)--> $sp,  Demand: $d, Reward = $r")
end


From 7 --(a=0)--> 5,  Demand: 2, Reward = -10
From 7 --(a=0)--> 0,  Demand: 9, Reward = -100
From 7 --(a=0)--> 3,  Demand: 4, Reward = -6
From 7 --(a=0)--> 0,  Demand: 7, Reward = 0
From 7 --(a=0)--> 0,  Demand: 7, Reward = 0
From 7 --(a=0)--> 0,  Demand: 7, Reward = 0
From 7 --(a=0)--> 0,  Demand: 10, Reward = -150
From 7 --(a=0)--> 0,  Demand: 8, Reward = -50
From 7 --(a=0)--> 5,  Demand: 2, Reward = -10
From 7 --(a=0)--> 6,  Demand: 1, Reward = -12


In [3]:
import Pkg; Pkg.add("TabularTDLearning")
using POMDPs
using QuickPOMDPs
using POMDPTools
using TabularTDLearning
using Random

   Resolving package versions...
  No Changes to `C:\Users\mathj\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\mathj\.julia\environments\v1.11\Manifest.toml`


In [4]:
function train_q_learning_agent(mdp, alpha::Float64, epsilon::Float64; iterations=10_000)
    ql = QLearningSolver(
        max_iterations=iterations,
        alpha=alpha,
        epsilon=epsilon,
        gamma=mdp.discount
    )
    policy = solve(ql, mdp)
    return policy, ql.q
end

train_q_learning_agent (generic function with 1 method)

In [5]:
function evaluate_policy(mdp, policy; num_trials=100, max_steps=100)
    sim = RolloutSimulator(max_steps=max_steps)
    rewards = [simulate(sim, mdp, policy) for _ in 1:num_trials]
    return mean(rewards)
end

evaluate_policy (generic function with 1 method)

# The value iteration algorithm to find the optimal policy

In [ ]:
function value_iteration_gen(mdp; γ=POMDPs.discount(mdp), θ=1e-4, max_iter=10000, n_samples=100)
    states = collect(POMDPs.states(mdp))
    actions_per_state = Dict(s => collect(POMDPs.actions(mdp, s)) for s in states)

    V = Dict(s => 0.0 for s in states)
    π = Dict(s => first(actions_per_state[s]) for s in states)

    rng = Random.MersenneTwister(42)  # fixe pour reproductibilité

    for iter in 1:max_iter
        Δ = 0.0
        V_new = copy(V)

        for s in states
            v_old = V[s]
            best_value = -Inf
            best_action = nothing

            for a in actions_per_state[s]
                total = 0.0
                for _ in 1:n_samples
                    sp, r, _ = POMDPs.gen(mdp, s, a, rng)
                    total += r + γ * V[sp]
                end
                value = total / n_samples

                if value > best_value
                    best_value = value
                    best_action = a
                end
            end

            V_new[s] = best_value
            π[s] = best_action
            Δ = max(Δ, abs(v_old - best_value))
        end

        V = V_new
        if Δ < θ
            println("Convergence atteinte (Δ < θ = $θ).")
            break
        end
    end

    return V, π
end


value_iteration_gen (generic function with 1 method)

In [7]:
V_opt, π_opt = value_iteration_gen(mdp)
for s in sort(collect(keys(π_opt)))
    println("Stock $s → Action optimale : ", π_opt[s] == 1 ? "Commander" : "Ne rien faire")
end


Stock 0 → Action optimale : Commander
Stock 1 → Action optimale : Commander
Stock 2 → Action optimale : Commander
Stock 3 → Action optimale : Commander
Stock 4 → Action optimale : Commander
Stock 5 → Action optimale : Commander
Stock 6 → Action optimale : Commander
Stock 7 → Action optimale : Commander
Stock 8 → Action optimale : Commander
Stock 9 → Action optimale : Commander
Stock 10 → Action optimale : Commander
Stock 11 → Action optimale : Commander
Stock 12 → Action optimale : Commander
Stock 13 → Action optimale : Ne rien faire
Stock 14 → Action optimale : Ne rien faire
Stock 15 → Action optimale : Ne rien faire
Stock 16 → Action optimale : Ne rien faire
Stock 17 → Action optimale : Ne rien faire
Stock 18 → Action optimale : Ne rien faire
Stock 19 → Action optimale : Ne rien faire
Stock 20 → Action optimale : Ne rien faire


The linear programming formulation to find the optimal policy